In [2]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
nltk.download('snowball_data')
nltk.download('stopwords')

[nltk_data] Downloading package snowball_data to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package snowball_data is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
stop_list=set(stopwords.words('french'))

In [4]:
def stem_lst(fichier):
    with open(fichier,'r') as file:
        content=file.read()
    content=content.lower()
    tokens=re.split(r"\W",content)
    mots=[t for t in tokens if not(re.match(r"([0-9]+-?)+",t)) and t not in stop_list]
    stemmer=SnowballStemmer("french")
    stems=[stemmer.stem(mot) for mot in mots]
    return stems

In [5]:
def TF(fichier):
    stems=stem_lst(fichier)
    dictionnaire={}
    for item in set(stems):
        dictionnaire[item]=stems.count(item)
    return dictionnaire

In [6]:
def dictionnaire_global():
        liste_doc=['C:\\Users\\user\\OneDrive\\Desktop\\SEMESTRE_2\\indexation\\corpus\\doc'+str(i)+'.txt' for i in range(1,11)]
        dic1={}
        for fich in liste_doc:
            dic1[fich]=TF(fich)
        return dic1

def IDF(terme):
    N=0
    for i in ['C:\\Users\\user\\OneDrive\\Desktop\\SEMESTRE_2\\indexation\\corpus\\doc'+str(i)+'.txt' for i in range(1,11)]:
        stems=stem_lst(i)
        if terme in stems:
            N+=1
            break
    return 1/N
def poids(stem,doc):
    return TF(doc)[stem]*IDF(stem)

In [7]:
def fichier_inverse():
    rExp=r"doc[0-9]+"
    with open('fichier-inverse.txt', 'w') as f:
        dg=dictionnaire_global()
        for fich in dg.keys():
            for stem in dg[fich].keys():
                match=re.search(rExp,fich)
                ligne=stem+'---'+match.group(0)+'---'+str(poids(stem,fich))+'\n'
                f.write(ligne)
    with open('fichier-inverse.txt','r') as file:
        content=file.read()
        print(content)
fichier_inverse()
def stemming_requete(requete):
  req=requete.lower()
  tokens=re.split(r"\W",req)
  mots=[t for t in tokens if not(re.match(r"([0-9]+-?)+",t)) and t not in stop_list]
  stemmer=SnowballStemmer("french")
  stems=[stemmer.stem(mot) for mot in mots]
  return stems

---doc1---7.0
or---doc1---1.0
a---doc1---2.0
peu---doc1---1.0
dor---doc1---1.0
bout---doc1---1.0
échos---doc1---1.0
trop---doc1---1.0
fer---doc1---1.0
juillet---doc1---1.0
papi---doc1---2.0
mer---doc1---1.0
rescap---doc1---1.0
terr---doc1---2.0
ru---doc1---1.0
hui---doc1---1.0
grand---doc1---2.0
apatrid---doc1---1.0
esclav---doc1---2.0
couteau---doc1---2.0
autrefois---doc1---1.0
adolescent---doc1---1.0
embauch---doc1---1.0
rosi---doc1---1.0
dépatri---doc1---1.0
tunisien---doc1---1.0
boît---doc1---1.0
brûleur---doc1---1.0
bien---doc1---1.0
bastill---doc1---1.0
souti---doc1---1.0
soleil---doc1---1.0
enfant---doc1---3.0
kabyl---doc1---1.0
ébouillanteur---doc1---1.0
retourn---doc1---1.0
jol---doc1---1.0
étranger---doc1---1.0
cigar---doc1---1.0
désoeuvr---doc1---1.0
indochinois---doc1---1.0
allé---doc1---1.0
capital---doc1---1.0
soir---doc1---1.0
bêt---doc1---1.0
pris---doc1---1.0
barcelon---doc1---1.0
polack---doc1---1.0
libert---doc1---1.0
local---doc1---1.0
visag---doc1---1.0
vit---doc1-

In [8]:
def TF_requete(requete):
    dictionnaire={}
    req=stemming_requete(requete)
    for item in set(req):
        dictionnaire[item]=req.count(item)
    return dictionnaire

In [17]:
def RSV(doc,requete):
  import math
  req=stemming_requete(requete)
  dict_p={}
  nominateur=0
  #extraire les mots d'un document a partir de son num et leurs poids
  with open('fichier-inverse.txt','r') as file:
        content=file.read()
        lines = content.splitlines()
        for line in lines:
          if re.match(r"^[a-z]+"+"---"+"doc"+str(doc)+r"---[0-9].[0-9]",line):
            mot=re.search(r"[a-z]+",line).group()
            dict_p[mot]=re.search(r"[0-9]+.[0-9]",line).group()
            #print(dict_p)
  #calcul de la regle de nominateur : poids du terme dans le document
  #* poids du meme terme dans la requete
  for term in req:
    if term in dict_p.keys():
      nominateur+=float(dict_p[term])*1
  sqrt_p_doc=0
  sum_p_doc=0
  #calcul de la somme des poids du doc
  for poids in dict_p.values():
    sum_p_doc+=float(poids)**2
  ##print(sum_p_doc)
  sqrt_p_doc=math.sqrt(sum_p_doc)
  #calcul de la somme des poids de la requete
  sum_p_requete= sum(map(lambda x: x**2, TF_requete(requete).values()))
  sqrt_p_requete=math.sqrt(sum_p_requete)
  RSV=(nominateur)/((sqrt_p_requete)*(sqrt_p_doc))
  return [("doc"+str(doc),RSV)]
RSV(1,"colon si")

def RSV_corpus_cosinus(requete,indice):
  RSV_corpus=[RSV(doc,requete) for doc in range(1,11)]
  sorted_RSV = sorted(RSV_corpus, key=lambda x: x[0][1], reverse=True)
  print("les "+str(indice) +" documents les plus pertinants selon la requete , metrique de cosinus")
  for i in range(0,indice):
    print(sorted_RSV[i])
RSV_corpus_cosinus(" la couleure du soleil est blue",10)

les 10 documents les plus pertinants selon la requete , metrique de cosinus
[('doc3', 0.06401843996644799)]
[('doc1', 0.037582301400141446)]
[('doc9', 0.02535100632816969)]
[('doc2', 0.0)]
[('doc4', 0.0)]
[('doc5', 0.0)]
[('doc6', 0.0)]
[('doc7', 0.0)]
[('doc8', 0.0)]
[('doc10', 0.0)]
